In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


In [2]:
df = pd.read_csv('data/df_with_pca.csv')

## Variabile dummy per Macroarea

In [3]:
df = pd.get_dummies(df, columns=["Macroarea"], drop_first=True)

## Variabile dummy per Descrizione Tipologia di Immobile e Renaming delle colonne per eliminare gli spazi

In [4]:
df = pd.get_dummies(df, columns=["Descrizione Tipologia di Immobile"], drop_first=True)

In [5]:
df.rename(columns={"Descrizione Tipologia di Immobile_Abitazioni di tipo economico": "Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico"}, inplace=True)
df.rename(columns={"Descrizione Tipologia di Immobile_Abitazioni signorili" : "Descrizione_Tipologia_di_Immobile_Abitazioni_signorili"}, inplace=True)
df.rename(columns={"Descrizione Tipologia di Immobile_Abitazioni tipiche dei luoghi": "Descrizione_Tipologia_di_Immobile_Abitazioni_tipiche_dei_luoghi"}, inplace=True)
df.rename(columns={"Descrizione Tipologia di Immobile_Ville e Villini": "Descrizione_Tipologia_di_Immobile_Ville_e_Villini"}, inplace=True)

## Eliminazione colonna Circoscrizione Turistica

In [6]:
df.drop(columns=["Circoscrizione Turistica"], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21513 entries, 0 to 21512
Data columns (total 52 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id_istat                                                         21513 non-null  int64  
 1   id_comune                                                        21512 non-null  object 
 2   Provincia                                                        21513 non-null  object 
 3   Regione                                                          21513 non-null  object 
 4   Ripartizione Geografica                                          21513 non-null  object 
 5   id_catastale                                                     21513 non-null  object 
 6   Indice di spopolamento                                           21512 non-null  float64
 7   Codice regione                          

Remember to convert the "object" variables into numeric so we can study relationships also in between these values

In [8]:
# show me the "Media Costo alla Vendita" column
print(df['Media Costo alla Vendita'])

0        0
1        0
2        0
3        0
4        0
        ..
21508    0
21509    0
21510    0
21511    0
21512    0
Name: Media Costo alla Vendita, Length: 21513, dtype: int64


In [9]:
# sobstitute the "Media Costo alla Vendita" column with the mean of the column Costo Massimo alla Vendita and Costo Minimo alla Vendita
df['Media Costo alla Vendita'] = (df['Costo Massimo alla Vendita'] + df['Costo Minimo alla Vendita']) / 2


In [10]:
# Split the data into training and test sets
train, test = train_test_split(df, test_size=0.2, random_state=1)


In [11]:
# Parameters estimation with statsmodels
model1 = smf.ols('Q("Media Costo alla Vendita") ~ Q("Indice di occupazione") + PC_Economica', train).fit()
model1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     Q("Media Costo alla Vendita")   R-squared:                       0.192
Model:                                       OLS   Adj. R-squared:                  0.192
Method:                            Least Squares   F-statistic:                     2040.
Date:                           Mon, 12 May 2025   Prob (F-statistic):               0.00
Time:                                   18:45:43   Log-Likelihood:            -1.3949e+05
No. Observations:                          17209   AIC:                         2.790e+05
Df Residuals:                              17206   BIC:                         2.790e+05
Df Model:                                      2                                         
Covariance Type:                       nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   -514.6163     39.427    -13.052      0.000    -591.898    -437.335
Q("Indice di occupazione")    39.1800      0.849     46.164      0.000      37.516      40.844
PC_Economica                  69.6542      1.662     41.906      0.000      66.396      72.912
==============================================================================
Omnibus:                    13527.515   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           483280.109
Skew:                           3.501   Prob(JB):                         0.00
Kurtosis:                      27.999   Cond. No.                         300.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Parameters estimation with statsmodels
model1 = smf.ols('Q("Media Costo alla Locazione") ~ PC_Turistica_1 + PC_Turistica_2 + PC_Turistica_3 + PC_Turistica_4 + PC_Turistica_5 + PC_Turistica_6 + PC_Turistica_7 + PC_Turistica_8', train).fit()
model1.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     Q("Media Costo alla Locazione")   R-squared:                       0.281
Model:                                         OLS   Adj. R-squared:                  0.280
Method:                              Least Squares   F-statistic:                     838.7
Date:                             Mon, 12 May 2025   Prob (F-statistic):               0.00
Time:                                     18:45:43   Log-Likelihood:                -40284.
No. Observations:                            17210   AIC:                         8.059e+04
Df Residuals:                                17201   BIC:                         8.065e+04
Df Model:                                        8                                         
Covariance Type:                         nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          4.0281      0.019    210.144      0.000       3.991       4.066
PC_Turistica_1     0.4410      0.006     76.763      0.000       0.430       0.452
PC_Turistica_2     0.0363      0.007      5.054      0.000       0.022       0.050
PC_Turistica_3     0.1990      0.009     22.213      0.000       0.181       0.217
PC_Turistica_4     0.1337      0.011     12.057      0.000       0.112       0.155
PC_Turistica_5    -0.0744      0.013     -5.857      0.000      -0.099      -0.049
PC_Turistica_6    -0.0027      0.014     -0.194      0.847      -0.031       0.025
PC_Turistica_7     0.0984      0.015      6.553      0.000       0.069       0.128
PC_Turistica_8     0.0159      0.016      1.014      0.311      -0.015       0.047
==============================================================================
Omnibus:                     6000.372   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           111945.222
Skew:                           1.199   Prob(JB):                         0.00
Kurtosis:                      15.262   Cond. No.                         3.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
df.columns


Index(['id_istat', 'id_comune', 'Provincia', 'Regione',
       'Ripartizione Geografica', 'id_catastale', 'Indice di spopolamento',
       'Codice regione', 'Capoluogo_enc', 'Codice comuni',
       'Indice di occupazione', 'Indice disoccupazione', 'Fascia',
       'Descrizione Zona', 'Sottofascia Zona', 'id_univoco',
       'Costo Minimo alla Vendita', 'Costo Massimo alla Vendita',
       'Media Costo alla Vendita', 'Costo Minimo alla Locazione',
       'Costo Massimo alla Locazione', 'Media Costo alla Locazione',
       'Cod. Circoscrizione Turistica',
       'Tipo di località_Altri comuni non altrimenti classificati',
       'Tipo di località_Capoluogo senza specifici interessi turistici',
       'Tipo di località_Città d'Arte', 'Tipo di località_Località collinari',
       'Tipo di località_Località lacuali', 'Tipo di località_Località marine',
       'Tipo di località_Località montane',
       'Tipo di località_Località religiose',
       'Tipo di località_Località termali', 'PC_Ec

In [14]:
# Parameters estimation with statsmodels
model2 = smf.ols('Q("Costo Minimo alla Vendita") ~ Q("Indice di spopolamento") + Q("Indice di occupazione") + Q("Indice disoccupazione") + Q("Tipo di località_Altri comuni non altrimenti classificati") + Q("Tipo di località_Capoluogo senza specifici interessi turistici") + Q("Tipo di località_Città d\'Arte") + Q("Tipo di località_Località collinari") + Q("Tipo di località_Località lacuali") + Q("Tipo di località_Località marine") + Q("Tipo di località_Località montane") + Q("Tipo di località_Località religiose") + Q("Tipo di località_Località termali") + PC_Economica + PC_Turistica_1 + PC_Turistica_2 + PC_Turistica_3 + PC_Turistica_4 + PC_Turistica_5 + PC_Turistica_6 + PC_Turistica_7 + PC_Turistica_8 + PC_Turistica_9 + PC_Turistica_10 + PC_Turistica_11 + PC_Turistica_12 + Macroarea_Isole + Macroarea_Nord + Macroarea_Sud + Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico + Descrizione_Tipologia_di_Immobile_Abitazioni_signorili + Descrizione_Tipologia_di_Immobile_Abitazioni_tipiche_dei_luoghi + Descrizione_Tipologia_di_Immobile_Ville_e_Villini', data=train).fit()
model2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     Q("Costo Minimo alla Vendita")   R-squared:                       0.512
Model:                                        OLS   Adj. R-squared:                  0.511
Method:                             Least Squares   F-statistic:                     563.5
Date:                            Mon, 12 May 2025   Prob (F-statistic):               0.00
Time:                                    18:45:45   Log-Likelihood:            -1.3228e+05
No. Observations:                           17207   AIC:                         2.646e+05
Df Residuals:                               17174   BIC:                         2.649e+05
Df Model:                                      32                                         
Covariance Type:                        nonrobust                                         
===============================================================================================================================================
                                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    -129.2217     67.804     -1.906      0.057    -262.124       3.680
Q("Tipo di località_Altri comuni non altrimenti classificati")[T.True]        -26.5988     20.305     -1.310      0.190     -66.399      13.202
Q("Tipo di località_Capoluogo senza specifici interessi turistici")[T.True]  -118.3214     32.185     -3.676      0.000    -181.408     -55.235
Q("Tipo di località_Città d'Arte")[T.True]                                   -187.4943     24.785     -7.565      0.000    -236.076    -138.912
Q("Tipo di località_Località collinari")[T.True]                              -96.8379     24.818     -3.902      0.000    -145.484     -48.192
Q("Tipo di località_Località lacuali")[T.True]                                184.7245     32.824      5.628      0.000     120.386     249.063
Q("Tipo di località_Località marine")[T.True]                                 335.8767     25.538     13.152      0.000     285.820     385.934
Q("Tipo di località_Località montane")[T.True]                                 22.4159     22.786      0.984      0.325     -22.246      67.078
Q("Tipo di località_Località religiose")[T.True]                             -556.4401    134.412     -4.140      0.000    -819.902    -292.979
Q("Tipo di località_Località termali")[T.True]                               -202.7019     39.273     -5.161      0.000    -279.682    -125.722
Macroarea_Isole[T.True]                                                        19.4071     21.951      0.884      0.377     -23.619      62.433
Macroarea_Nord[T.True]                                                        -67.2097     10.758     -6.247      0.000     -88.296     -46.123
Macroarea_Sud[T.True]                                                          26.4945     16.305      1.625      0.104      -5.464      58.453
Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico[T.True]       -303.9271     11.766    -25.831      0.000    -326.990    -280.864
Descrizione_Tipologia_di_Immobile_Abitazioni_signorili[T.True]                717.7946    181.360      3.958      0.000     362.310    1073.279
Descrizione_Tipologia_di_Immobile_Abitazioni_tipiche_dei_luoghi[T.True]      -606.4042    528.810     -1.147      0.252   -1642.926     430.117
Descrizione_Tipologia_di_Immobile_Ville_e_Villini[T.True]                     344.9535     21.803     15.821      0.000     302.217     387.690
Q("Indice di spopolamento")                                            

# Ripetiamo la regressione facendo feature selection ed eliminando le variabili con il p-value > 0.05

In [15]:
model2 = smf.ols('Q("Costo Minimo alla Vendita") ~ Q("Indice di spopolamento") + Q("Indice di occupazione") + Q("Indice disoccupazione") + Q("Tipo di località_Capoluogo senza specifici interessi turistici") + Q("Tipo di località_Città d\'Arte") + Q("Tipo di località_Località collinari") + Q("Tipo di località_Località lacuali") + Q("Tipo di località_Località marine") + Q("Tipo di località_Località religiose") + Q("Tipo di località_Località termali") + PC_Economica + PC_Turistica_1 + PC_Turistica_2 + PC_Turistica_3 + PC_Turistica_4 + PC_Turistica_5 + PC_Turistica_6 + PC_Turistica_7 + PC_Turistica_8 + PC_Turistica_9 + PC_Turistica_10 + PC_Turistica_11 + PC_Turistica_12 + Macroarea_Nord + Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico + Descrizione_Tipologia_di_Immobile_Abitazioni_signorili + Descrizione_Tipologia_di_Immobile_Ville_e_Villini', data=train).fit()
model2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     Q("Costo Minimo alla Vendita")   R-squared:                       0.512
Model:                                        OLS   Adj. R-squared:                  0.511
Method:                             Least Squares   F-statistic:                     667.0
Date:                            Mon, 12 May 2025   Prob (F-statistic):               0.00
Time:                                    18:45:45   Log-Likelihood:            -1.3229e+05
No. Observations:                           17207   AIC:                         2.646e+05
Df Residuals:                               17179   BIC:                         2.648e+05
Df Model:                                      27                                         
Covariance Type:                        nonrobust                                         
===============================================================================================================================================
                                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    -116.1586     64.069     -1.813      0.070    -241.741       9.424
Q("Tipo di località_Capoluogo senza specifici interessi turistici")[T.True]  -102.0711     26.396     -3.867      0.000    -153.811     -50.332
Q("Tipo di località_Città d'Arte")[T.True]                                   -177.6280     16.909    -10.505      0.000    -210.771    -144.485
Q("Tipo di località_Località collinari")[T.True]                              -86.1436     16.769     -5.137      0.000    -119.013     -53.274
Q("Tipo di località_Località lacuali")[T.True]                                194.4224     27.413      7.092      0.000     140.690     248.155
Q("Tipo di località_Località marine")[T.True]                                 345.5455     17.296     19.978      0.000     311.644     379.447
Q("Tipo di località_Località religiose")[T.True]                             -556.7797    133.039     -4.185      0.000    -817.550    -296.009
Q("Tipo di località_Località termali")[T.True]                               -194.9116     34.987     -5.571      0.000    -263.490    -126.333
Macroarea_Nord[T.True]                                                        -72.8481     10.401     -7.004      0.000     -93.235     -52.462
Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico[T.True]       -300.4031     11.723    -25.625      0.000    -323.382    -277.424
Descrizione_Tipologia_di_Immobile_Abitazioni_signorili[T.True]                716.1911    181.310      3.950      0.000     360.804    1071.578
Descrizione_Tipologia_di_Immobile_Ville_e_Villini[T.True]                     344.9769     21.786     15.835      0.000     302.274     387.679
Q("Indice di spopolamento")                                                     1.3066      0.392      3.332      0.001       0.538       2.075
Q("Indice di occupazione")                                                     28.8826      1.102     26.210      0.000      26.723      31.043
Q("Indice disoccupazione")                                                     -4.0667      1.277     -3.185      0.001      -6.570      -1.564
PC_Economica                                                                   32.6495      4.994      6.538      0.000      22.861      42.438
PC_Turistica_1                                                                 95.6851      3.472     27.559      0.000      88.880     102.491
PC_Turistica_2                                                         

In [16]:
# creating a new df with all the variables we need
df2 = df[['id_istat', 'id_comune', 'Provincia', 'Regione',
       'Ripartizione Geografica', 'id_catastale', 'Indice di spopolamento',
       'Codice regione', 'Capoluogo_enc', 'Codice comuni',
       'Indice di occupazione', 'Indice disoccupazione', 'Fascia',
       'Descrizione Zona', 'Sottofascia Zona', 'id_univoco',
       'Costo Minimo alla Vendita', 'Costo Massimo alla Vendita',
       'Media Costo alla Vendita', 'Costo Minimo alla Locazione',
       'Costo Massimo alla Locazione', 'Media Costo alla Locazione',
       'Cod. Circoscrizione Turistica',
       'Tipo di località_Altri comuni non altrimenti classificati',
       'Tipo di località_Capoluogo senza specifici interessi turistici',
       'Tipo di località_Città d\'Arte', 'Tipo di località_Località collinari',
       'Tipo di località_Località lacuali', 'Tipo di località_Località marine',
       'Tipo di località_Località montane',
       'Tipo di località_Località religiose',
       'Tipo di località_Località termali', 'PC_Economica', 'PC_Turistica_1',
       'PC_Turistica_2', 'PC_Turistica_3', 'PC_Turistica_4', 'PC_Turistica_5',
       'PC_Turistica_6', 'PC_Turistica_7', 'PC_Turistica_8', 'PC_Turistica_9',
       'PC_Turistica_10', 'PC_Turistica_11', 'PC_Turistica_12',
       'Macroarea_Isole', 'Macroarea_Nord', 'Macroarea_Sud',
       'Descrizione_Tipologia_di_Immobile_Abitazioni_di_tipo_economico',
       'Descrizione_Tipologia_di_Immobile_Abitazioni_signorili',
       'Descrizione_Tipologia_di_Immobile_Abitazioni_tipiche_dei_luoghi',
       'Descrizione_Tipologia_di_Immobile_Ville_e_Villini']]


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21513 entries, 0 to 21512
Data columns (total 52 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id_istat                                                         21513 non-null  int64  
 1   id_comune                                                        21512 non-null  object 
 2   Provincia                                                        21513 non-null  object 
 3   Regione                                                          21513 non-null  object 
 4   Ripartizione Geografica                                          21513 non-null  object 
 5   id_catastale                                                     21513 non-null  object 
 6   Indice di spopolamento                                           21512 non-null  float64
 7   Codice regione                          

In [18]:
#convert the new df into a csv and insert it into the data folder
df2.to_csv('data/df_regression.csv', index=False)

OSError: [Errno 22] Invalid argument: 'data/df_regression.csv'